## Reading in API key from your environment

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    print("API key not found")
elif not api_key.startswith('sk-proj-'):
    print("An API key was found but it doesn't start with sk-proj-")
elif api_key.strip() != api_key:
    print("An API key was found but it contain whitespace before or end of it")
else:
    print("API key found")

API key found


## Start using OpenAI

### Resources

https://platform.openai.com/docs/quickstart?api-mode=responses&lang=python

### Connecting to OpenAI

In [5]:
client = OpenAI()

### Calling client for text generation

In [ ]:
message = "Hello, GPT!"
response = client.responses.create(
    model='gpt-4o-mini',
    input=message
)
print(response.output_text)

### Analyze image inputs

In [6]:
response = client.responses.create(
    model='gpt-4o-mini',
    input=[
        {"role": "user", "content": "what team is playing in this image?"},
        {
            "role": "user",
            "content": [
                {
                    "type": "input_image",
                    "image_url": "https://upload.wikimedia.org/wikipedia/commons/3/3b/LeBron_James_Layup_%28Cleveland_vs_Brooklyn_2018%29.jpg"
                }
            ]
        }
    ]
)

print(response.output_text)

The image shows a basketball game featuring players from the Cleveland Cavaliers and the Brooklyn Nets.


## Business applications

### Summarizing website - W1D1

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# get api key
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    print("API key not found")
elif not api_key.startswith('sk-proj-'):
    print("An API key was found but it doesn't start with sk-proj-")
elif api_key.strip() != api_key:
    print("An API key was found but it contain whitespace before or end of it")
else:
    print("API key found")

# load client
client = OpenAI()

API key found


#### A class to represent a webpage

In [ ]:
from bs4 import BeautifulSoup
import requests

# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

#### A function to return user prompt

In [24]:
def user_prompt_for(website: Website):
    user_prompt = f"You are looking at a webiste titled: {website.title}"
    user_prompt += "\nThe content for the website is as follows;\
    please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

#### System prompt

In [26]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

#### A function to return input for GPT-4o-mini

In [33]:
def inputs_for(url: str):
    website = Website(url=url)
    return [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt_for(website)}
    ]

#### A function to put it all together

In [36]:
from IPython.display import Markdown, display

def summarize(url: str):
    response = client.responses.create(
        input=inputs_for(url=url),
        model='gpt-4o-mini'
    )
    display(Markdown(response.output_text))

In [37]:
summarize('https://www.medium.com')

# Summary of Medium

Medium is a platform dedicated to reading and writing stories, fostering a community where users can share and explore diverse ideas and narratives. The platform emphasizes deepening understanding through various human stories. 

Key Features:
- **Membership**: Offers exclusive content for subscribers.
- **Writing Opportunities**: Encourages users to contribute their own stories and perspectives.
- **Reading**: Users can start reading articles on a wide range of topics.

Overall, Medium serves as a hub for both content consumers and creators, promoting engagement with a variety of ideas. 

No specific news or announcements were mentioned in the provided content.